In [1]:
#https://keras.io/examples/vision/image_classification_efficientnet_fine_tuning/
from tensorflow.keras.applications import EfficientNetB0

In [2]:
import tensorflow as tf 
import pathlib 

def load_train_val_data(image_size=(384, 512), batch_size=16):
    base_dir = pathlib.Path('..')
    data_train = base_dir / 'data' / 'train'
    print('Loading data')
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '../data/train',
        validation_split=0.2,
        subset='training',
        label_mode='categorical',
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_train,
        validation_split=0.2,
        subset='validation',
        label_mode='categorical',
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    return train_ds, val_ds

In [3]:
train_ds, val_ds = load_train_val_data(image_size=(224,224))

Loading data
Found 1766 files belonging to 6 classes.
Using 1413 files for training.
Found 1766 files belonging to 6 classes.
Using 353 files for validation.


In [6]:
IMG_SIZE = 224
from tensorflow.keras import layers
img_augmentation = tf.keras.models.Sequential(
    [
        layers.RandomRotation(factor=0.15),
        layers.RandomTranslation(height_factor=0.1, width_factor=0.1),
        layers.RandomFlip(),
        layers.RandomContrast(factor=0.1),
    ],
    name="img_augmentation",
)

In [7]:
from tensorflow.keras import layers

def build_model(num_classes):
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
    x = img_augmentation(inputs)
    model = EfficientNetB0(include_top=False, input_tensor=inputs, weights="imagenet")

    # Freeze the pretrained weights
    model.trainable = False

    # Rebuild top
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)

    top_dropout_rate = 0.2
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(800, activation="relu", name="pred")(x)
    outputs = layers.Dense(600, activation="softmax", name="pred")(x)
    outputs = layers.Dense(100, activation="softmax", name="pred")(x)
    outputs = layers.Dense(num_classes, activation="softmax", name="pred")(x)

    # Compile
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(
        optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"]
    )
    return model

In [8]:
model = build_model(num_classes=6)

In [9]:
model.summary()

Model: "EfficientNet"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 rescaling (Rescaling)          (None, 224, 224, 3)  0           ['input_1[0][0]']                
                                                                                                  
 normalization (Normalization)  (None, 224, 224, 3)  7           ['rescaling[0][0]']              
                                                                                                  
 stem_conv_pad (ZeroPadding2D)  (None, 225, 225, 3)  0           ['normalization[0][0]'

In [ ]:
from matplotlib import pyplot as plt
def plot_hist(hist):
    plt.plot(hist.history["accuracy"])
    plt.plot(hist.history["val_accuracy"])
    plt.title("model accuracy")
    plt.ylabel("accuracy")
    plt.xlabel("epoch")
    plt.legend(["train", "validation"], loc="upper left")
    plt.show()

In [ ]:
#tf.keras.utils.plot_model(model, show_shapes=True)
model = build_model(num_classes=6)

In [37]:
epochs = 5
hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)

Epoch 1/5
89/89 - 27s - loss: 1.4343 - accuracy: 0.7183 - val_loss: 0.8002 - val_accuracy: 0.7394 - 27s/epoch - 303ms/step
Epoch 2/5
89/89 - 18s - loss: 0.7602 - accuracy: 0.8436 - val_loss: 0.7885 - val_accuracy: 0.7904 - 18s/epoch - 202ms/step
Epoch 3/5
89/89 - 18s - loss: 0.8869 - accuracy: 0.8351 - val_loss: 0.9491 - val_accuracy: 0.8159 - 18s/epoch - 200ms/step
Epoch 4/5
89/89 - 18s - loss: 0.6891 - accuracy: 0.8754 - val_loss: 1.2008 - val_accuracy: 0.7904 - 18s/epoch - 200ms/step
Epoch 5/5
89/89 - 18s - loss: 0.6103 - accuracy: 0.8825 - val_loss: 1.1810 - val_accuracy: 0.8300 - 18s/epoch - 200ms/step


# Fine tuning model 

In [3]:
from tensorflow import keras
model = keras.models.load_model('../my_scripts/models/EfficientNet/simple_665_0.89.h5')

In [4]:
model.trainable = True
model.compile(optimizer=keras.optimizers.Adam(1e-5),
loss='categorical_crossentropy', metrics=['accuracy'])

In [7]:
import tensorflow as tf
def load_train_val_data(image_size=(384, 512), batch_size=16):
    base_dir = pathlib.Path('..')
    data_train = base_dir / 'data' / 'train'
    print('Loading data')
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        '../data/train',
        validation_split=0.2,
        subset='training',
        label_mode='categorical',
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )

    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        data_train,
        validation_split=0.2,
        subset='validation',
        label_mode='categorical',
        seed=1337,
        image_size=image_size,
        batch_size=batch_size,
    )
    return train_ds, val_ds

import pathlib
train_ds, val_ds = load_train_val_data(image_size=(224,224))

Loading data
Found 1766 files belonging to 6 classes.
Using 1413 files for training.
Found 1766 files belonging to 6 classes.
Using 353 files for validation.


In [8]:
model.fit(train_ds, epochs=5, validation_data=val_ds, verbose=2)

Epoch 1/5


ResourceExhaustedError:  OOM when allocating tensor with shape[10752,17,17] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node EfficientNet/block6a_dwconv_pad/Pad
 (defined at C:\Users\fkhan\.virtualenvs\Garbage_classification-zsuLYxXv\lib\site-packages\keras\backend.py:3672)
]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info. This isn't available when running in Eager mode.
 [Op:__inference_train_function_17733]

Errors may have originated from an input operation.
Input Source operations connected to node EfficientNet/block6a_dwconv_pad/Pad:
In[0] EfficientNet/block6a_expand_activation/IdentityN (defined at C:\Users\fkhan\.virtualenvs\Garbage_classification-zsuLYxXv\lib\site-packages\keras\activations.py:270)	
In[1] EfficientNet/block6a_dwconv_pad/Pad/paddings:

Operation defined at: (most recent call last)
>>>   File "c:\users\fkhan\appda